In [2]:
%load_ext autoreload
import cobra
import numpy as np
from importlib import reload
import mcs
from scipy import sparse
#import traceback
#import warnings
import sys


This is a bilevel-example with 1 Target and 1 Desired region

In [15]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
mcs.fba(network,constr=["r1=0"])
# specify modules
# specify modules
modules = [mcs.SD_Module(network,"mcs_bilvl",module_sense="target",constraints=["r_P <= 0.5"],inner_objective="-r_BM")]
modules.append(mcs.SD_Module(network,"mcs_lin",module_sense="desired",constraints="r_BM >= 1"))

# specify MCS setup
ko_cost = {'r2': 1, 'r3': 1, 'r4': 1, 'r5': 1, 'r6': 1, 'r7': 1, 'r8': 1, 'r9': 1}
ki_cost = {'r1': 1}
maxSolutions = 50
maxCost = 50
solver = 'gurobi'

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,
       ko_cost=ko_cost,ki_cost=ki_cost,max_cost=maxCost,M=None,
       solver=solver)

# solve MILP
mcsEnum.enumerate()
# mcsEnum.compute()
# mcsEnum.compute_optimal()

Constructing MCS MILP.
Bounding MILP.
Enumerating strain designs ...
Strain designs with cost 4.0: [{'r1': 1.0, 'r4': -1.0, 'r7': -1.0, 'r9': -1.0}]
Strain designs with cost 4.0: [{'r1': 1.0, 'r5': -1.0, 'r7': -1.0, 'r9': -1.0}]
Finished. 2 solutions found.


([{'r1': 1.0, 'r4': -1.0, 'r7': -1.0, 'r9': -1.0},
  {'r1': 1.0, 'r5': -1.0, 'r7': -1.0, 'r9': -1.0}],
 0)

This is a bilevel-example with only 1 desired region

In [16]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
mcs.fba(network,constr=["r1=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="desired",constraints=["r_BM >= 0.5","r_P >= 1"],inner_objective="-r_BM")

# specify MCS setup
ko_cost = {'r2': 1, 'r3': 1, 'r4': 1, 'r5': 1, 'r6': 1, 'r7': 1, 'r8': 1, 'r9': 1}
ki_cost = {'r1': 1}
maxSolutions = 50
maxCost = 50

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=7,M=None)

# solve MILP
mcsEnum.compute_optimal()

Constructing MCS MILP.
Bounding MILP.
Finding optimal strain designs ...
Strain design with cost 3: {'r1': 1, 'r5': -1, 'r7': -1}
Strain design with cost 3: {'r1': 1, 'r4': -1, 'r7': -1}
Finished. 2 solutions found.


([{'r1': 1.0, 'r5': -1.0, 'r7': -1.0}, {'r1': 1.0, 'r4': -1.0, 'r7': -1.0}], 0)

This is a regular MCS example with 1 target and 1 desired region

In [17]:
%autoreload
# load network
network = cobra.io.read_sbml_model("SmallExample.sbml")

# specify modules
modules = [mcs.SD_Module(network,"mcs_lin",module_sense="target",constraints=["R04 >= 1","2 R01 >= 0.5"])]
modules.append(mcs.SD_Module(network,"mcs_lin",module_sense="desired",constraints="R03 >= 1"))

# specify MCS setup
ko_cost = {'R05': 2, 'R06': 3, 'R07': 4, 'R08': 4, 'R09': 4, 'R10': 4}
ki_cost = {'R02': 6}
maxSolutions = 8
maxCost = 8

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=maxCost,M=None)

# solve MILP
mcsEnum.enumerate()
# mcsEnum.compute()
# mcsEnum.compute_optimal()

Constructing MCS MILP.
Bounding MILP.
Enumerating strain designs ...
Strain designs with cost 4.0: [{'R07': -1.0}]
Strain designs with cost 4.0: [{'R10': -1.0}]
Strain designs with cost 7.0: [{'R06': -1.0, 'R08': -1.0}]
Finished. 3 solutions found.


([{'R07': -1.0}, {'R10': -1.0}, {'R06': -1.0, 'R08': -1.0}], 0)

This is a bilevel-example with 1 Target and 1 Desired region

In [37]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")
mcs.fba(network,constr=["r1=0"])
# specify modules
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="target",constraints=["r_P <= 0.5"],inner_objective="-r_BM")

# specify MCS setup
maxSolutions = 8
maxCost = 3
solver = 'gurobi'

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,max_cost=maxCost,M=None,solver=solver)

# solve MILP
# mcsEnum.compute()
# mcsEnum.compute(max_solutions=maxSolutions)
# mcsEnum.enumerate(max_solutions=maxSolutions)
mcsEnum.enumerate(max_solutions=maxSolutions)
# mcsEnum.compute_optimal(max_solutions=maxSolutions)

This is a mid-scale-example with only 1 desired region

In [20]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0

sol = mcs.fba(network,constr=["O2Up=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_bilvl",module_sense="desired",constraints=["2 Growth >= 0.2","EthEx >= 1"],inner_objective="-Growth")

network.remove_metabolites
network.metabolites.list_attr("compartment")
# specify MCS setup
maxSolutions = 8
maxCost = 4
solver = 'gurobi'

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,max_cost=maxCost,solver=solver,M=None)

# solve MILP
mcsEnum.enumerate(max_solutions=maxSolutions)
# mcsEnum.compute_optimal(max_solutions=maxSolutions,time_limit=5)
# mcsEnum.compute(max_solutions=maxSolutions,time_limit=5)

Constructing MCS MILP.
Bounding MILP.
Enumerating strain designs ...
Strain designs with cost 1.0: [{'O2Up': -1.0}]
Strain designs with cost 1.0: [{'CYTBO3_4pp': -1.0}]
Strain designs with cost 1.0: [{'h2oEx': -1.0}]
Strain designs with cost 3.0: [{'ATPS4rpp': -1.0, 'SUCCt3pp': -1.0, 'THD2pp': -1.0}]
Strain designs with cost 3.0: [{'MDH': -1.0, 'PPS': -1.0, 'TPI': -1.0}]
Strain designs with cost 3.0: [{'AcEx': -1.0, 'ATPS4rpp': -1.0, 'SUCOAS': -1.0}]
Strain designs with cost 3.0: [{'ATPS4rpp': -1.0, 'PTAr': -1.0, 'SUCOAS': -1.0}]
Strain designs with cost 3.0: [{'ACKr': -1.0, 'ATPS4rpp': -1.0, 'SUCOAS': -1.0}]
Finished. 8 solutions found.


([{'O2Up': -1.0},
  {'CYTBO3_4pp': -1.0},
  {'h2oEx': -1.0},
  {'ATPS4rpp': -1.0, 'SUCCt3pp': -1.0, 'THD2pp': -1.0},
  {'MDH': -1.0, 'PPS': -1.0, 'TPI': -1.0},
  {'AcEx': -1.0, 'ATPS4rpp': -1.0, 'SUCOAS': -1.0},
  {'ATPS4rpp': -1.0, 'PTAr': -1.0, 'SUCOAS': -1.0},
  {'ACKr': -1.0, 'ATPS4rpp': -1.0, 'SUCOAS': -1.0}],
 0)

In [21]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0
network.reactions.ATPM.upper_bound = 1000
network.reactions.ATPM.lower_bound = 3.15

sol = mcs.fba(network,constr=["O2Up=0"])
# specify modules
modules = mcs.SD_Module(network,"mcs_lin",module_sense="target")

network.remove_metabolites
network.metabolites.list_attr("compartment")
# specify MCS setup
maxSolutions = 4
maxCost = 50

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,max_cost=7,M=None)

# solve MILP
mcsEnum.compute_optimal(max_solutions=maxSolutions)

Constructing MCS MILP.
Bounding MILP.
Finding optimal strain designs ...
Strain design with cost 1: {'ATPM': -1}
Strain design with cost 2: {'CO2Ex': -1, 'h_pEx': -1}
Strain design with cost 2: {'ENO': -1, 'O2Up': -1}
Strain design with cost 2: {'GlcUp': -1, 'SUCDi': -1}
Finished. 4 solutions found.


([{'ATPM': -1.0},
  {'CO2Ex': -1.0, 'h_pEx': -1.0},
  {'ENO': -1.0, 'O2Up': -1.0},
  {'GlcUp': -1.0, 'SUCDi': -1.0}],
 0)

In [23]:
%autoreload
# load network
network = cobra.io.read_sbml_model("ECC2comp.sbml")
# remove external metabolites and obsolete reactions
external_mets = [i for i,cpts in zip(network.metabolites,network.metabolites.list_attr("compartment")) if cpts == 'External_Species']
network.remove_metabolites(external_mets)
S = cobra.util.create_stoichiometric_matrix(network)
obsolete_reacs = [reac for reac,b_rempty in zip(network.reactions,np.any(S,0)) if not b_rempty]
network.remove_reactions(obsolete_reacs)
# close some uptakes
network.reactions.AcUp.upper_bound = 0
network.reactions.SuccUp.upper_bound = 0
network.reactions.GlycUp.upper_bound = 0
network.reactions.CO2Up = 0
network.reactions.ATPM.upper_bound = 1000
network.reactions.ATPM.lower_bound = 3.15

# sol = mcs.fba(network,constr=["O2Up=0"])
# sol1 = mcs.fba(network,constr=['AcEx = 0','ATPS4rpp = 0', 'G3PD5= 0', 'NADTRHD= 0', 'PFL= 0', 'PYK= 0', 'SuccEx= 0', 'SUCCt2_2pp= 0'])
# sol2 = mcs.fva(network,constr=['Growth = '+str(0.999*sol1.objective_value),'AcEx = 0','ATPS4rpp = 0', 'G3PD5= 0', 'NADTRHD= 0', 'PFL= 0', 'PYK= 0', 'SuccEx= 0', 'SUCCt2_2pp= 0'])

# specify modules
modules = mcs.SD_Module(network,"optknock",constraints=["Growth >= 0.2","EthEx >= 1"],inner_objective="-Growth",outer_objective="-EthEx")

# specify MCS setup
maxSolutions = 100
maxCost = 2
solver = 'cplex'

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,max_cost=maxCost,M=None, solver=solver)

# solve MILP
# mcsEnum.compute(max_solutions=maxSolutions)
# mcsEnum.compute_optimal(max_solutions=maxSolutions)
mcsEnum.enumerate(max_solutions=maxSolutions)

Constructing MILP.
Bounding MILP.
Enumerating strain designs ...
Enumerating all solutions with the objective value -13.868049181833047
Strain designs with cost 2.0: [{'AcEx': -1.0, 'O2Up': -1.0}]
Enumerating all solutions with the objective value -13.868049181833076
Strain designs with cost 2.0: [{'AcEx': -1.0, 'CYTBO3_4pp': -1.0}]
Enumerating all solutions with the objective value -13.776295937549136
Strain designs with cost 2.0: [{'CYTBO3_4pp': -1.0, 'PTAr': -1.0}]
Strain designs with cost 2.0: [{'O2Up': -1.0, 'PTAr': -1.0}]
Strain designs with cost 2.0: [{'ACKr': -1.0, 'CYTBO3_4pp': -1.0}]
Enumerating all solutions with the objective value -13.776295937549136
Strain designs with cost 2.0: [{'ACKr': -1.0, 'O2Up': -1.0}]
Enumerating all solutions with the objective value -11.602860050576798
Strain designs with cost 2.0: [{'AcEx': -1.0, 'h2oEx': -1.0}]
Enumerating all solutions with the objective value -11.599739089700504
Strain designs with cost 2.0: [{'h2oEx': -1.0, 'THD2pp': -1.0}]

([{'AcEx': -1.0, 'O2Up': -1.0},
  {'AcEx': -1.0, 'CYTBO3_4pp': -1.0},
  {'CYTBO3_4pp': -1.0, 'PTAr': -1.0},
  {'O2Up': -1.0, 'PTAr': -1.0},
  {'ACKr': -1.0, 'CYTBO3_4pp': -1.0},
  {'ACKr': -1.0, 'O2Up': -1.0},
  {'AcEx': -1.0, 'h2oEx': -1.0},
  {'h2oEx': -1.0, 'THD2pp': -1.0},
  {'ACKr': -1.0, 'h2oEx': -1.0},
  {'h2oEx': -1.0, 'PTAr': -1.0},
  {'ATPS4rpp': -1.0, 'h2oEx': -1.0},
  {'O2Up': -1.0, 'THD2pp': -1.0},
  {'CYTBO3_4pp': -1.0, 'THD2pp': -1.0},
  {'O2Up': -1.0, 'PPC': -1.0},
  {'CYTBO3_4pp': -1.0, 'PPC': -1.0},
  {'h2oEx': -1.0, 'TKT2': -1.0},
  {'h2oEx': -1.0, 'RPE': -1.0},
  {'h2oEx': -1.0, 'SUCOAS': -1.0},
  {'h2oEx': -1.0, 'TKT1': -1.0},
  {'h2oEx': -1.0, 'TALA': -1.0},
  {'h2oEx': -1.0},
  {'CYTBO3_4pp': -1.0, 'PYK': -1.0},
  {'O2Up': -1.0, 'PYK': -1.0},
  {'O2Up': -1.0, 'TKT2': -1.0},
  {'CYTBO3_4pp': -1.0, 'TKT2': -1.0},
  {'FHL': -1.0, 'O2Up': -1.0},
  {'h2Ex': -1.0, 'O2Up': -1.0},
  {'CYTBO3_4pp': -1.0, 'FHL': -1.0},
  {'CYTBO3_4pp': -1.0, 'h2Ex': -1.0},
  {'CO2Ex': -1.0